In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloaders.train import BalancedMelSampler
from dataloaders.val import MelSampler
from dataloaders.utils import csv_to_dict

from losses import NpairsLoss, MovingAverageBCE
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks
from models import NUM_FRAMES, Classifier, DeepMetricLearning

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def get_lr(epoch, lr, min_lr=1e-5, max_lr=1e-3, epochs=30, warmup_prop=0.1):   
    if epoch <= epochs * warmup_prop:
        return min_lr + max_lr - (max_lr * (epochs * warmup_prop - epoch) / (epochs * warmup_prop))
    else:
        return min_lr + max_lr - (max_lr * (epoch - epochs * warmup_prop) / (epochs - epochs * warmup_prop))

In [9]:
def main(model_name, batch_size=64, fold_idx=0, saved_path=""):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = csv_to_dict(train_data.iloc[train_index])
    fold_valid_dict = csv_to_dict(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=batch_size,
        max_length=NUM_FRAMES,
        n_classes=24,
        use_cutmix=True,
        n_classes_in_batch=8,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        max_length=NUM_FRAMES,
    )

    print(' -> Preparing Model \n')
    model = get_model(
        model_name=model_name,
        saved_path=saved_path,
        pretrained_with_contrastive=True,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=50,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    step_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    ma_bce = MovingAverageBCE(
        train_data.iloc[train_index],
        start_apply_step=20 * step_per_epoch,
        momentum=0.9,
        name="moving_average_loss",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        moving_average_bce=ma_bce,
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(True, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
log_folder = "../logs/"

In [11]:
MODELS = [
#     'densenet121',
    'resnet18',
    'resnet34',
    'resnext50',
    'efficientnetb2',
    "xception",
]

BIGGER_MODELS = [  # BS = 32
    'resnest50',
    'efficientnetb3',
    'efficientnetb4',
]

In [12]:
# for model_name in MODELS:
#     print(f"\n\nModel {model_name}\n\n")
#     if not DEBUG:
#         log_folder = prepare_log_folder(LOG_PATH)
#         print(f'Logging results to {log_folder}')
#         create_logger(directory=log_folder, name="logs.txt")

#     for fold_idx in range(5):
#         tf.keras.backend.clear_session()
#         print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")

#         main(
#             model_name=model_name,
#             fold_idx=fold_idx, 
#             saved_path=log_folder, 
#         )

#         if DEBUG:
#             break

In [ ]:
for model_name in BIGGER_MODELS:
    print(f"\n\nModel {model_name}\n\n")
    if not DEBUG:
        log_folder = prepare_log_folder(LOG_PATH)
        print(f'Logging results to {log_folder}')
        create_logger(directory=log_folder, name="logs.txt")

    for fold_idx in range(5):
        tf.keras.backend.clear_session()
        print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")

        main(
            model_name=model_name,
            batch_size=32,
            fold_idx=fold_idx, 
            saved_path=log_folder, 
        )

        if DEBUG:
            break



Model resnest50


Logging results to ../logs/2021-02-14/5/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)
 -> Training Model 

Epoch 1/100
Instructions for updating:
Use `tf.linalg.matmul` instead
30/30 - 69s - loss: 3.3110 - val_loss: 2.8909
Epoch 2/100
30/30 - 21s - loss: 3.1938 - val_loss: 2.8896
Epoch 3/100
30/30 - 20s - loss: 3.2388 - val_loss: 2.8894
Epoch 4/100
30/30 - 21s - loss: 3.2055 - val_loss: 2.8898
Epoch 5/100
30/30 - 19s - loss: 3.2609 - val_loss: 2.8871
Epoch 6/100
30/30 - 20s - loss: 3.2508 - val_loss: 2.8885
Epoch 7/100
30/30 - 20s - loss: 3.1942 - val_loss: 2.8723
Epoch 8/100
30/30 - 20s - loss: 2.7872 - val_loss: 2.7859
Epoch 9/100
30/30 - 20s - loss: 3.4776 - val_loss: 2.8867
Epoch 10/100
30/30 - 20s - loss: 3.3472 - val_loss: 2.8884
Epoch 11/100
30/30 - 20s - loss: 3.0007 - val_loss: 2.8717
Epoch 12/100
30/30 - 20s - loss: 2.8805 - val_loss: 2.4181
Epoc

Epoch 62/100
30/30 - 19s - loss: 1.8565 - val_loss: 1.4888
Epoch 63/100
30/30 - 20s - loss: 3.1207 - val_loss: 1.4947
Epoch 64/100
30/30 - 20s - loss: 3.1668 - val_loss: 1.5397
Epoch 65/100
30/30 - 20s - loss: 2.0076 - val_loss: 1.4086
Epoch 66/100
30/30 - 21s - loss: 3.2863 - val_loss: 1.3672
Epoch 67/100
30/30 - 20s - loss: 1.8772 - val_loss: 1.4034
Epoch 68/100
30/30 - 20s - loss: 3.0551 - val_loss: 1.4136
Epoch 69/100
30/30 - 20s - loss: 2.1908 - val_loss: 1.4551
Epoch 70/100
30/30 - 20s - loss: 1.9699 - val_loss: 1.5009
Epoch 71/100
30/30 - 20s - loss: 1.9134 - val_loss: 1.4644
Epoch 72/100
30/30 - 20s - loss: 1.7883 - val_loss: 1.3953
Epoch 73/100
30/30 - 20s - loss: 1.8758 - val_loss: 1.3715
Epoch 74/100
30/30 - 20s - loss: 1.6945 - val_loss: 1.4073
Epoch 75/100
30/30 - 21s - loss: 3.1759 - val_loss: 1.5107
Epoch 76/100
30/30 - 20s - loss: 2.9506 - val_loss: 1.4918
Epoch 77/100
30/30 - 20s - loss: 1.8199 - val_loss: 1.4041
Epoch 78/100
30/30 - 20s - loss: 2.9808 - val_loss: 1.51

Epoch 10/100
30/30 - 20s - loss: 2.8863 - val_loss: 2.7477
Epoch 11/100
30/30 - 20s - loss: 2.8436 - val_loss: 2.6290
Epoch 12/100
30/30 - 20s - loss: 2.8662 - val_loss: 2.2926
Epoch 13/100
30/30 - 20s - loss: 2.4992 - val_loss: 1.7995
Epoch 14/100
30/30 - 20s - loss: 2.3318 - val_loss: 1.6091
Epoch 15/100
30/30 - 20s - loss: 2.6348 - val_loss: 1.6616
Epoch 16/100
30/30 - 20s - loss: 2.4925 - val_loss: 1.6356
Epoch 17/100
30/30 - 20s - loss: 2.4974 - val_loss: 1.3785
Epoch 18/100
30/30 - 21s - loss: 2.4260 - val_loss: 1.4755
Epoch 19/100
30/30 - 20s - loss: 2.4603 - val_loss: 1.4561
Epoch 20/100
30/30 - 20s - loss: 2.6333 - val_loss: 1.4325
Epoch 21/100
30/30 - 20s - loss: 2.9861 - val_loss: 1.3833
Epoch 22/100
30/30 - 20s - loss: 2.5403 - val_loss: 1.2966
Epoch 23/100
30/30 - 20s - loss: 2.4156 - val_loss: 1.2685
Epoch 24/100
30/30 - 20s - loss: 2.2199 - val_loss: 1.3540
Epoch 25/100
30/30 - 20s - loss: 2.6713 - val_loss: 1.1877
Epoch 26/100
30/30 - 20s - loss: 3.2216 - val_loss: 1.32

Epoch 43/100
30/30 - 20s - loss: 2.8568 - val_loss: 1.1440
Epoch 44/100
30/30 - 20s - loss: 1.9283 - val_loss: 1.0329
Epoch 45/100
30/30 - 20s - loss: 2.8184 - val_loss: 1.0939
Epoch 46/100
30/30 - 20s - loss: 2.6984 - val_loss: 1.0812
Epoch 47/100
30/30 - 20s - loss: 1.7228 - val_loss: 1.1560
Epoch 48/100
30/30 - 19s - loss: 1.7727 - val_loss: 0.9510
Epoch 49/100
30/30 - 20s - loss: 2.9896 - val_loss: 0.9151
Epoch 50/100
30/30 - 20s - loss: 1.4775 - val_loss: 0.9310
Epoch 51/100
30/30 - 20s - loss: 3.0051 - val_loss: 1.0517
Epoch 52/100
30/30 - 20s - loss: 1.7546 - val_loss: 1.1535
Epoch 53/100
30/30 - 20s - loss: 1.6070 - val_loss: 1.0584
Epoch 54/100
30/30 - 20s - loss: 1.6623 - val_loss: 1.1268
Epoch 55/100
30/30 - 20s - loss: 1.8267 - val_loss: 1.0132
Epoch 56/100
30/30 - 20s - loss: 3.1949 - val_loss: 1.1308
Epoch 57/100
30/30 - 20s - loss: 1.6796 - val_loss: 0.9802
Epoch 58/100
30/30 - 19s - loss: 3.0436 - val_loss: 0.8645
Epoch 59/100
30/30 - 20s - loss: 1.6897 - val_loss: 0.91

Epoch 82/100
30/30 - 20s - loss: 3.1947 - val_loss: 1.8193
Epoch 83/100
30/30 - 20s - loss: 2.9985 - val_loss: 1.5248
Epoch 84/100
30/30 - 20s - loss: 1.5071 - val_loss: 1.5587
Epoch 85/100
30/30 - 20s - loss: 1.5798 - val_loss: 1.4019
Epoch 86/100
30/30 - 20s - loss: 1.4291 - val_loss: 1.5326
Epoch 87/100
30/30 - 20s - loss: 2.9859 - val_loss: 1.6713
Epoch 88/100
30/30 - 20s - loss: 2.9962 - val_loss: 1.6272
Epoch 89/100
30/30 - 20s - loss: 2.9292 - val_loss: 1.7081
Epoch 90/100
30/30 - 19s - loss: 2.9627 - val_loss: 1.7719
Epoch 91/100
30/30 - 20s - loss: 1.5303 - val_loss: 2.2854
Epoch 92/100
30/30 - 19s - loss: 2.8844 - val_loss: 2.1786
Epoch 93/100
30/30 - 20s - loss: 3.0179 - val_loss: 1.8013
Epoch 94/100
30/30 - 20s - loss: 1.5075 - val_loss: 1.5484
Epoch 95/100
30/30 - 20s - loss: 1.5741 - val_loss: 1.6764
Epoch 96/100
30/30 - 20s - loss: 2.9645 - val_loss: 1.3837
Epoch 97/100
30/30 - 19s - loss: 2.7429 - val_loss: 1.4236
Epoch 98/100
30/30 - 20s - loss: 1.4793 - val_loss: 1.38

Epoch 11/100
30/30 - 20s - loss: 2.5081 - val_loss: 3.2185
Epoch 12/100
30/30 - 20s - loss: 3.2638 - val_loss: 3.2189
Epoch 13/100
30/30 - 20s - loss: 3.2004 - val_loss: 3.2197
Epoch 14/100
30/30 - 19s - loss: 2.4110 - val_loss: 3.2183
Epoch 15/100
30/30 - 20s - loss: 2.9137 - val_loss: 3.2177
Epoch 16/100
30/30 - 20s - loss: 2.6283 - val_loss: 3.2139
Epoch 17/100
30/30 - 20s - loss: 2.3666 - val_loss: 3.2022
Epoch 18/100
30/30 - 20s - loss: 3.2797 - val_loss: 3.1733
Epoch 19/100
30/30 - 20s - loss: 2.8735 - val_loss: 3.1368
Epoch 20/100
30/30 - 20s - loss: 2.3263 - val_loss: 3.0312
Epoch 21/100
30/30 - 19s - loss: 2.4088 - val_loss: 2.9818
Epoch 22/100
30/30 - 20s - loss: 2.1806 - val_loss: 2.8838
Epoch 23/100
30/30 - 20s - loss: 2.3502 - val_loss: 2.6114
Epoch 24/100
30/30 - 20s - loss: 2.3685 - val_loss: 2.7018
Epoch 25/100
30/30 - 20s - loss: 2.1384 - val_loss: 2.6058
Epoch 26/100
30/30 - 20s - loss: 1.8691 - val_loss: 2.4830
Epoch 27/100
30/30 - 19s - loss: 3.1689 - val_loss: 2.58

Epoch 82/100
30/30 - 19s - loss: 1.5106 - val_loss: 1.0391
Epoch 83/100
30/30 - 20s - loss: 1.4836 - val_loss: 1.0226
Epoch 84/100
30/30 - 19s - loss: 2.9671 - val_loss: 1.1121
Epoch 85/100
30/30 - 19s - loss: 2.8499 - val_loss: 1.2263
Epoch 86/100
30/30 - 20s - loss: 2.9816 - val_loss: 1.2637
Epoch 87/100
30/30 - 19s - loss: 2.7744 - val_loss: 1.2791
Epoch 88/100
30/30 - 20s - loss: 1.7883 - val_loss: 1.1618
Epoch 89/100
30/30 - 19s - loss: 1.4692 - val_loss: 1.1274
Epoch 90/100
30/30 - 20s - loss: 2.6575 - val_loss: 1.3161
Epoch 91/100
30/30 - 19s - loss: 1.7567 - val_loss: 1.0204
Epoch 92/100
30/30 - 19s - loss: 1.5810 - val_loss: 1.1727
Epoch 93/100
30/30 - 20s - loss: 2.6900 - val_loss: 1.0962
Epoch 94/100
30/30 - 20s - loss: 3.0894 - val_loss: 1.0206
Epoch 95/100
30/30 - 19s - loss: 2.8565 - val_loss: 0.9111
Epoch 96/100
30/30 - 20s - loss: 2.8898 - val_loss: 0.9074
Epoch 97/100
30/30 - 19s - loss: 2.8695 - val_loss: 0.8521
Epoch 98/100
30/30 - 19s - loss: 1.6172 - val_loss: 0.89

Epoch 9/100
30/30 - 27s - loss: 3.4034 - val_loss: 2.8907
Epoch 10/100
30/30 - 27s - loss: 3.0928 - val_loss: 2.8907
Epoch 11/100
30/30 - 27s - loss: 2.5204 - val_loss: 2.8905
Epoch 12/100
30/30 - 26s - loss: 3.1500 - val_loss: 2.8907
Epoch 13/100
30/30 - 26s - loss: 2.5533 - val_loss: 2.8902
Epoch 14/100
30/30 - 27s - loss: 3.2046 - val_loss: 2.8901
Epoch 15/100
30/30 - 26s - loss: 3.1417 - val_loss: 2.8903
Epoch 16/100
30/30 - 26s - loss: 2.5562 - val_loss: 2.8882
Epoch 17/100
30/30 - 27s - loss: 2.9254 - val_loss: 2.8819
Epoch 18/100
30/30 - 26s - loss: 2.0153 - val_loss: 2.8704
Epoch 19/100
30/30 - 26s - loss: 3.1325 - val_loss: 2.7947
Epoch 20/100
30/30 - 27s - loss: 2.2908 - val_loss: 2.7160
Epoch 21/100
30/30 - 26s - loss: 2.9715 - val_loss: 2.6425
Epoch 22/100
30/30 - 26s - loss: 3.0920 - val_loss: 2.5824
Epoch 23/100
30/30 - 27s - loss: 3.2032 - val_loss: 2.5025
Epoch 24/100
30/30 - 27s - loss: 2.4465 - val_loss: 2.4480
Epoch 25/100
30/30 - 26s - loss: 3.2138 - val_loss: 2.253